In [51]:

import os
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_redis import RedisVectorStore
from langchain.docstore.document import Document
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 가져오기
REDIS_URL = os.getenv('REDIS_URL')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [54]:
import os
import time
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_redis import RedisVectorStore
from langchain.docstore.document import Document
from dotenv import load_dotenv
import redis

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 가져오기
REDIS_URL = os.getenv('REDIS_URL')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Redis 클라이언트 설정
r = redis.Redis(host='localhost', port=6379, db=0, password=os.getenv('REDIS_PASSWORD'))

# OpenAI Embeddings 설정
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# RedisVectorStore 설정 (embedding_function 대신 embeddings 객체를 전달)
redis_vector_store = RedisVectorStore(redis_url=REDIS_URL, embeddings=embeddings)

# OpenAI LLM 설정
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# 문서를 Redis에 저장하는 함수 (장기 기억에 저장)
def save_document_to_redis(text, metadata=None, long_term=True):
    # 문서 생성
    document = Document(page_content=text, metadata=metadata)

    # 장기 기억에 저장
    redis_vector_store.add_documents([document])
    print(f"장기🥫 기억에 저장됨: {text}")

# 문서를 단기 기억에 저장하는 함수
def save_short_term_document(key, value, ttl_seconds=20):
    # 데이터를 저장하고 TTL 설정 (20초 후 자동 삭제)
    r.set(key, value)
    r.expire(key, ttl_seconds)
    print(f"단기🥩 기억에 저장됨: {key} = {value} (TTL: {ttl_seconds}초)")

# Query Routing: 사용자의 쿼리에 따라 장기 기억과 단기 기억을 분리
def query_routing(query):
    # "망고밍밍" 키워드가 포함되면 장기 기억에 저장
    if "망고밍밍" in query:
        save_document_to_redis(f"질문: {query}", metadata={'id': f"long_term_{int(time.time())}"}, long_term=True)
    
    # 다른 키워드일 경우 단기 기억에 저장
    else:
        save_short_term_document(f"short_term_{int(time.time())}", f"질문: {query}", ttl_seconds=20)

    # 질의에 대한 답변을 얻기 위해 Redis에서 검색
    retriever = redis_vector_store.as_retriever()

    # RetrievalQA 체인 설정
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever
    )

    # 질의에 대한 답변 가져오기
    response = qa_chain.run(query)
    return response

# 사용자와의 대화 예시: 기억을 기반으로 질문과 답변을 이어나가도록 설계
def long_term_memory_conversation(query):
    print("😀질문:", query)
    
    # 1. 쿼리 라우팅: 장기 기억과 단기 기억을 구분하여 처리
    response = query_routing(query)
    
    # 2. 대답 출력
    print("답변:", response)

long_term_memory_conversation("내가 좋아하는 과일은 망고밍밍이야.")  # 이 문장은 장기 기억에 저장

long_term_memory_conversation("서울의 날씨는 어때?")  # 이 문장은 단기 기억에 저장되고 20초 후 자동 삭제

long_term_memory_conversation("망고밍밍은 여름에 만들어진 과일이지")  # "오대산"에 대한 정보가 장기 기억에서 불러와짐

long_term_memory_conversation("과일 추천해줄래?")  # 이 문장은 단기 기억에 저장되고 20초 후 자동 삭제

long_term_memory_conversation("망고밍밍은 뭐야?")  # 장기 기억에서 오대산에 대한 정보 불러옴


😀질문: 내가 좋아하는 과일은 망고밍밍이야.
장기🥫 기억에 저장됨: 질문: 내가 좋아하는 과일은 망고밍밍이야.
답변:  망고밍밍은 과일이 아닙니다.
😀질문: 서울의 날씨는 어때?
단기🥩 기억에 저장됨: short_term_1732523947 = 질문: 서울의 날씨는 어때? (TTL: 20초)
답변:  서울의 날씨는 현재 맑고 따뜻합니다.
😀질문: 망고밍밍은 여름에 만들어진 과일이지
장기🥫 기억에 저장됨: 질문: 망고밍밍은 여름에 만들어진 과일이지
답변:  네, 망고밍밍은 여름에 만들어진 과일입니다.
😀질문: 과일 추천해줄래?
단기🥩 기억에 저장됨: short_term_1732523950 = 질문: 과일 추천해줄래? (TTL: 20초)
답변:  아직 여름이니까 망고밍밍을 추천할게요! 다른 과일들도 맛있지만 망고밍밍이 제일 좋아요. 
😀질문: 망고밍밍은 뭐야?
장기🥫 기억에 저장됨: 질문: 망고밍밍은 뭐야?
답변:  망고밍밍은 과일의 이름이다.


In [56]:
import redis
from dotenv import load_dotenv
import os
import time

# .env 파일에서 환경 변수 로드
load_dotenv()

# Redis 서버의 비밀번호를 .env 파일에서 불러오기
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# Redis 클라이언트 설정 (비밀번호 추가)
r = redis.Redis(host='localhost', port=6379, db=0, password=REDIS_PASSWORD)


keys = r.keys('*')
print("만료 전 모든 키:", keys)

# 예시: "단기기억" 키 저장 및 TTL 설정

def check_ttl(key):
    ttl = r.ttl(key)  # 키에 대한 TTL 값을 확인
    if ttl == -2:
        return "키가 존재하지 않음 (이미 삭제됨)"
    elif ttl == -1:
        return "키에 TTL이 설정되지 않음"
    else:
        return f"남은 TTL: {ttl}초"

# 데이터 저장 후 TTL 확인
save_short_term_document("short_term_key", "단기 기억 내용")

# 30초 후 TTL 값 확인
time.sleep(30)
status = check_ttl("short_term_key")
print(status)  # TTL이 만료되면 '키가 존재하지 않음 (이미 삭제됨)' 출력


# 다시 모든 키 조회
keys = r.keys('*')
print("만료 후 모든 키:", keys)


만료 전 모든 키: [b'01JDH7EPYXP7PXMT4ARBGTA8VB:a21eb4a124b34373bc89694ba1c77be9', b'01JDH7EPYXP7PXMT4ARBGTA8VB:1cf5201d036246389d7a4ad23fc04c8e', b'01JDH7JTG7K5Y34H6E1EEXE8P6:15096d91609b488cb59d4b9913f6cd25', b'01JDH7EPYXP7PXMT4ARBGTA8VB:3d6f245554004120887414672d881550', b'01JDH7JTG7K5Y34H6E1EEXE8P6:845a9b89c7664043be083ba2cf6eae3c', b'01JDH7JTG7K5Y34H6E1EEXE8P6:431ed12542aa45ec805aefe9f0ab45e4']
단기🥩 기억에 저장됨: short_term_key = 단기 기억 내용 (TTL: 20초)
키가 존재하지 않음 (이미 삭제됨)
만료 후 모든 키: [b'01JDH7EPYXP7PXMT4ARBGTA8VB:a21eb4a124b34373bc89694ba1c77be9', b'01JDH7EPYXP7PXMT4ARBGTA8VB:1cf5201d036246389d7a4ad23fc04c8e', b'01JDH7JTG7K5Y34H6E1EEXE8P6:15096d91609b488cb59d4b9913f6cd25', b'01JDH7EPYXP7PXMT4ARBGTA8VB:3d6f245554004120887414672d881550', b'01JDH7JTG7K5Y34H6E1EEXE8P6:845a9b89c7664043be083ba2cf6eae3c', b'01JDH7JTG7K5Y34H6E1EEXE8P6:431ed12542aa45ec805aefe9f0ab45e4']
